In [ ]:
# Boto3 SageMaker Invoke Endpoint
# This example shows how to invoke SageMaker Endpoint from outside of AWS environment using Boto3 SDK
# Boto is the Amazon Web Services (AWS) SDK for Python
# https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

# Endpoint: XGBoost - Kaggle Bike Rental - Regressor Trained in XGBoost Lectures
# Makesure Endpoint is deployed before running this example
# 
# Reference:
#  https://github.com/awslabs/amazon-sagemaker-examples

# NOTE: SageMaker SDK now requires additional permissions DescribeEndpoint, DescribeEndpointConfig in-addition to InvokeEndpoint
#   boto3 SDK requires just InvokeEndpoint permission.
#   Please update SageMakerInvokeEndpoint permissions to reflect this policy document:
#   Logon with my_admin account and update permissions (IAM->Policies->SageMakerInvokeEndpoint->Edit Policy)
#   
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "sagemaker:DescribeEndpointConfig",
                "sagemaker:DescribeEndpoint",
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}

In [1]:
import boto3
import math
import dateutil
import re

In [2]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='ml_user_predict',region_name='us-east-1')

In [3]:
# List of low level clients that are available in boto3
print(boto_session.get_available_services())

['accessanalyzer', 'account', 'acm', 'acm-pca', 'alexaforbusiness', 'amp', 'amplify', 'amplifybackend', 'amplifyuibuilder', 'apigateway', 'apigatewaymanagementapi', 'apigatewayv2', 'appconfig', 'appconfigdata', 'appflow', 'appintegrations', 'application-autoscaling', 'application-insights', 'applicationcostprofiler', 'appmesh', 'apprunner', 'appstream', 'appsync', 'athena', 'auditmanager', 'autoscaling', 'autoscaling-plans', 'backup', 'backup-gateway', 'batch', 'billingconductor', 'braket', 'budgets', 'ce', 'chime', 'chime-sdk-identity', 'chime-sdk-meetings', 'chime-sdk-messaging', 'cloud9', 'cloudcontrol', 'clouddirectory', 'cloudformation', 'cloudfront', 'cloudhsm', 'cloudhsmv2', 'cloudsearch', 'cloudsearchdomain', 'cloudtrail', 'cloudwatch', 'codeartifact', 'codebuild', 'codecommit', 'codedeploy', 'codeguru-reviewer', 'codeguruprofiler', 'codepipeline', 'codestar', 'codestar-connections', 'codestar-notifications', 'cognito-identity', 'cognito-idp', 'cognito-sync', 'comprehend', 'com

In [4]:
# Acquire a SageMaker Runtime client for us-east-1 region
client = boto_session.client(service_name='sagemaker-runtime',region_name='us-east-1')

In [5]:
# Specify Your Endpoint Name
endpoint_name = 'xgboost-biketrain-v1'

In [6]:
# Raw Data
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [7]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [8]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

Raw Data:
 2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027
Transformed Data:
 4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17


In [9]:
# Let's invoke prediction now
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=transform_data(sample_one).encode('utf-8'),
                       ContentType='text/csv')

In [11]:
result = result['Body'].read().decode('utf-8')

In [12]:
print(result)

6.349300861358643



In [13]:
# Actual Count is 562...but predicted is 6.36.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
print ('Predicted Count', math.expm1(float(result)))

Predicted Count 571.092597122044


In [14]:
print('\n'.join([transform_data(sample_one),transform_data(sample_two)]))

4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17
4,0,1,1,15.58,19.695,50,23.9994,2012,12,19,2,18


In [15]:
# Prediction for multiple observations in the same call
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=('\n'.join([transform_data(sample_one),
                                        transform_data(sample_two)]).encode('utf-8')),
                       ContentType='text/csv')

In [16]:
result = result['Body'].read().decode('utf-8')

In [17]:
result

'6.349300861358643\n6.321451187133789\n'

In [18]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    # Splitting using regular expression as xgboost 1-2-2 is returning
    # predicted values with inconsistent delimiters (comma, newline or both)

    # pattern looks for one or more of non-numeric characters
    pattern = r'[^0-9.]+'

    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=('\n'.join(transformed_data[i : i + batch_size]).encode('utf-8')),
                       ContentType='text/csv')
        
        result = result['Body'].read().decode('utf-8')
        result = re.split(pattern,result)
                
        predictions += [math.expm1(float(r)) for r in result if r != ""]
                
    return predictions

In [19]:
run_predictions([sample_one,sample_two,sample_three],10)

0 10


[571.092597122044, 555.3798181158465, 10.489585991183136]

In [20]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [21]:
len(data)

6493

In [22]:
%%time
predictions = run_predictions(data,100)

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
2400 2500
2500 2600
2600 2700
2700 2800
2800 2900
2900 3000
3000 3100
3100 3200
3200 3300
3300 3400
3400 3500
3500 3600
3600 3700
3700 3800
3800 3900
3900 4000
4000 4100
4100 4200
4200 4300
4300 4400
4400 4500
4500 4600
4600 4700
4700 4800
4800 4900
4900 5000
5000 5100
5100 5200
5200 5300
5300 5400
5400 5500
5500 5600
5600 5700
5700 5800
5800 5900
5900 6000
6000 6100
6100 6200
6200 6300
6300 6400
6400 6500
Wall time: 7.42 s


In [23]:
len(predictions),len(data)

(6493, 6493)